# Part 1

In [1]:
import re
from sympy.utilities.iterables import multiset_permutations

In [2]:
test_input = """???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1
"""

In [3]:
def gen_string(groups, gaps):
    return ''.join(('.'*gap ) + ('#'*group) for group, gap in zip(groups+[0], gaps))

def test_match(pattern, test_string):
    if re.match(pattern.replace("?", "[#|.]").replace(".", "\."), test_string):
        return True
    else:
        return False
    
def post_to_num(post, n):
    out = [0]* n
    _i = 0
    for v in post:
        if v == '|':
            _i+= 1
        else:
            out[_i] += 1
    return out

In [4]:
with open("inputs/day12.txt", "r") as fn:
    input = fn.read()

In [5]:
def part1(input):
    lines = [l.split(" ") for l in input.splitlines()]
    overall_total = 0
    for pattern, groups in lines:
        groups = list(map(int, groups.split(",")))
        gaps = [0] + [1]*(len(groups)-1) + [0]
        dof = len(pattern) - sum(groups) - sum(gaps)
        count = 0
        total = 0
        for perm in multiset_permutations(["|"]*(len(gaps)-1) +  ["*"]*dof):
            _additions = post_to_num(perm, len(gaps))
            _gaps = [i+j for i,j in zip(gaps, _additions)]
            _string = gen_string(groups, _gaps)    
            if test_match(pattern, _string):
                count += 1
                overall_total += 1
            total+=1
    print(overall_total)

part1(input)

8180


# Part 2

In [6]:
from functools import cache
from tqdm.notebook import tqdm

In [8]:
@cache
def fork_off(pattern, groups, i, n_in_group, n_complete_seq):
      if i == len(pattern):
            return 1 if len(groups) == n_complete_seq else 0
      elif pattern[i] == '#':
            return fork_off(pattern, groups, i + 1, n_in_group + 1, n_complete_seq)
      elif pattern[i] == '.' or n_complete_seq == len(groups):
            if n_complete_seq < len(groups) and n_in_group == groups[n_complete_seq]:
                  return fork_off(pattern, groups, i + 1, 0, n_complete_seq + 1)
            elif n_in_group == 0:
                  return fork_off(pattern, groups, i + 1, 0, n_complete_seq)
            else:
                  return 0
      else:
            hash_fork = fork_off(pattern, groups, i + 1, n_in_group + 1, n_complete_seq)
            if n_in_group == groups[n_complete_seq]:
                  return hash_fork + fork_off(pattern, groups, i + 1, 0, n_complete_seq + 1)
            elif n_in_group == 0:
                  return hash_fork + fork_off(pattern, groups, i + 1, 0, n_complete_seq)
            return hash_fork

In [10]:
def part2(lines):
    lines = [l.split(" ") for l in input.splitlines()]
    expanded_lines = list()
    n = 5
    for line in lines:
        expanded_lines.append([
            "?".join([line[0]]*n),
            ",".join([line[1]]*n)
            ])
    
    overall_total = 0
    for pattern, groups in tqdm(expanded_lines):
        groups = list(map(int,groups.split(",")))
        overall_total += fork_off(pattern+'.', tuple(groups), 0, 0, 0)
    print(overall_total)

part2(input)

  0%|          | 0/1000 [00:00<?, ?it/s]

620189727003627
